Model the base depth using fbprophet, which is a GAM: additive model that sums continuous functions to create the predicted output.

In [ ]:
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
long_series_df = pd.read_pickle(all_data_path).fillna(0)  # TODO: fix fillna func in data cleaning notebook

In [3]:
def resample_by_month(df, time_col):
    """resample df on monthly basis using specified timestamp column"""
    
    # handling of time_col
    acceptable_time_col = ['timestamp', 'pseudo_ts']
    other_time_col = [x for x in acceptable_time_col if x != time_col].pop()
    if time_col not in acceptable_time_col:
        raise Exception(f'must be in {acceptable_time_col}')
    
    month = (df
             .set_index(f'{time_col}')
             .resample('M')
             .agg({'base': 'mean', 'snowfall': 'sum', 'station': 'first', 
                   'region': 'first', 'state': 'first', 'ski_yr': 'first',
                   f'{other_time_col}': 'first'})
            )
    return month


all_data_path = r'../../data/snow_data_clean.pkl'
long_series_df = pd.read_pickle(all_data_path)
month_ts_df = resample_by_month(long_series_df, 'timestamp')

In [ ]:
m = Prophet(mcmc_samples=300, changepoint_prior_scale=0.01, 
            seasonality_mode='multiplicative', yearly_seasonality=10,
            weekly_seasonality=True, daily_seasonality=False)

m.add_regressor('snowfall')
m.fit(df)

# made df including space for predictions
future = m.make_future_dataframe(periods=12, freq='M')
future.tail()

In [4]:
forecast = m.predict(future)
fig1 = m.plot(forecast)

NameError: name 'Prophet' is not defined

In [ ]:
fig2 = m.plot_components(forecast)
